# VacationPY

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import gmaps.datasets
import os

# Import API key
from api_key import google_key

Read from csv and store in dataframe

In [2]:
# Cities.csv file from WeatherPy folder
weather_df = pd.read_csv('../WeatherPy/output_data/cities.csv')
weather_df.head(5)

,city,cloudiness,Country,date,humid,lat,lng,max_temp,wind_speed
0,san patricio,12,MX,03/16/2020,84,19.22,-104.70,76.91,7.09
1,kodiak,40,US,03/16/2020,64,57.79,-152.41,39.20,9.17
2,rikitea,3,PF,03/16/2020,79,-23.12,-134.97,79.63,12.71
3,belaya gora,14,RU,03/16/2020,87,68.53,146.42,-10.98,1.70
4,dingle,0,PH,03/16/2020,56,11.00,122.67,84.79,11.03


Create Humidity Heat Map

In [3]:
# Setup google key

gmaps.configure(api_key = google_key)

In [4]:
# figure_layout = {
#     'width': '400px',
#     'height': '300px',
#     'border': '1px solid black',
#     'padding': '1px',
#     'margin': '0 auto 0 auto'
#     }
# # fig = gmaps.figure(layout = figure_layout)

In [5]:
# Setup location using lat an long 
locations = weather_df[['lat','lng']]
humidity = weather_df['humid']

fig = gmaps.figure(center = (10,-40), zoom_level = 2)
fig.add_layer(gmaps.heatmap_layer(locations, weights=humidity,
                                 max_intensity = 100, 
                                  point_radius = 5))
fig

Figure(layout=FigureLayout(height='420px'))

Add Criteria to dataframe

In [6]:
# Create new dataframe to use for criteria
weather_criteria_df = pd.DataFrame(weather_df, 
                                   columns =["city", 
                                             "Country",
                                             'lat',
                                             'lng',
                                             'max_temp',
                                             'wind_speed',
                                             'cloudiness',
                                             'humid'
])
# weather_criteria_df
max_temp = (weather_criteria_df['max_temp']<=75) &(weather_criteria_df['max_temp']>40)
low_humid = (weather_criteria_df['humid']<50)
low_wind_speed = (weather_criteria_df['wind_speed']<10)

ideal_vacation_df = weather_criteria_df[max_temp & low_humid & low_wind_speed]
ideal_vacation_df.count()

city          38
Country       38
lat           38
lng           38
max_temp      38
wind_speed    38
cloudiness    38
humid         38
dtype: int64

Find Hotel

In [7]:
# Add new column to store hotel name
ideal_vacation_df['hotel_name'] = ''
ideal_vacation_df.head(5)

C:\Users\JeabD\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,city,Country,lat,lng,max_temp,wind_speed,cloudiness,humid,hotel_name
5,xingyi,CN,24.19,102.82,73.36,6.46,58,49,
14,nurota,UZ,40.56,65.69,60.80,6.93,100,48,
63,sohbatpur,PK,28.52,68.54,70.63,4.54,35,35,
91,jalu,LY,29.03,21.55,64.78,6.44,0,26,
92,luban,PL,51.12,15.29,41.00,5.99,91,32,


In [8]:
vacation_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
hotel_search = 'hotel'
radius = 1000

params = {
    "keyword": hotel_search,
    "radius": radius,
    "key": google_key
}

# Loop through the df and search for hotels in specific radius
# Use iterrowsto iterate through pandas dataframe
for index, row in ideal_vacation_df.iterrows():
    
    # Get lat and long from dataframe
    lat = row['lat']
    long = row['lng']
    city = row['city']
    
    # Add location by collect lat and long from dataframe
    params['location'] = f'{lat},{long}'
#     print(f'{params}')
    
    # Make request from the google website
    vacation_hotels = requests.get(vacation_url, params = params).json()
  
    try:
        hotel_name = vacation_hotels['results'][0]['name']
        
        ideal_vacation_df.loc[index, 'hotel_name'] = hotel_name
        print(f"** Closest hotel in {city} is {hotel_name}.**")
#         print(f'*************')
    
    except IndexError:
        print(f'There is no Hotel near {city}.')
#         print(f'-------')
 

There is no Hotel near xingyi.


C:\Users\JeabD\Anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


** Closest hotel in nurota is Guest House Ruslan Nurata & Tours.**
** Closest hotel in sohbatpur is Kattah Hotel.**
** Closest hotel in jalu is Jalu Hotel.**
** Closest hotel in luban is Hotel Tęczowa.**
There is no Hotel near bilma.
There is no Hotel near magaria.
There is no Hotel near hami.
** Closest hotel in upington is The Maxton Boutique Hotel Upington.**
** Closest hotel in san juan is Civico Art Hotel.**
** Closest hotel in mount gambier is Commodore on the Park.**
There is no Hotel near najran.
There is no Hotel near aksu.
** Closest hotel in waw is Shalom Hotel.**
** Closest hotel in kemin is Hotel Kok-Oirok.**
** Closest hotel in neuquen is Hilton Garden Inn Neuquen.**
There is no Hotel near shebalino.
** Closest hotel in takanabe is Takanabe Daiichi Hotel.**
** Closest hotel in kharan is Balochistan Hotel & Rest.**
** Closest hotel in doume is Hotel Universel.**
** Closest hotel in kita is HÔTEL LE RELAIS.**
** Closest hotel in wuan is Xinyuan Hotel.**
There is no Hotel ne

In [9]:
ideal_vacation_df.head(5)
ideal_vacation_df.count()

city          38
Country       38
lat           38
lng           38
max_temp      38
wind_speed    38
cloudiness    38
humid         38
hotel_name    38
dtype: int64

In [14]:
# Select only row that have hotel name
hotel_df = ideal_vacation_df.loc[ideal_vacation_df['hotel_name'] != '']
# hotel_df.head(38)
hotel_df.count()

city          25
Country       25
lat           25
lng           25
max_temp      25
wind_speed    25
cloudiness    25
humid         25
hotel_name    25
dtype: int64

In [17]:
# Create heat map with interactive markers
hotel_markers = hotel_df[['lat','lng']]
hotel_locations = [hotel_markers for hotel in hotel_df]
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel_name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
# hotel_info = [info_box_template.format(**hotel) for index, hotel in hotel_df]
marker_layer = gmaps.marker_layer(hotel_markers, info_box_content = hotel_info)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))